## Cross Validation
**Cross validation** is a technique used to evaluate the performance of a predictive model and assess its accuracy. It works by dividing the dataset into training and testing sets, then using the training set to build the model and the testing set to evaluate the model’s performance. This process is repeated multiple times with different combinations of training and testing sets, and the results are averaged to get an overall accuracy score. Cross validation helps ensure that the model is not overfitting or underfitting the data.

In [3]:
# Required libraries...
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()

In [4]:
# To split the dataset into train and test samples:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size = 0.2)

In [5]:
# Applying Logisctic Regression Classifire:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\Habib\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9722222222222222

In [6]:
# Applying Support Vector Machine Classifier: 
svm = SVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.9861111111111112

In [8]:
# Applying Random Forest Classifier:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9722222222222222

* So this is the quick result of three algorithms. As we see we split the data: 80% is using for training and 20% is using for testing. But this splitting is not uniform, if we execute it again and again it will change the samples and finally will brings changes in the models performance. So this is the key problem with train_test_split method that you can't run the model once and judge whether my model is performing better then other models. You must run it multiple times.
* Now let's in coming part check K-fold cross validation.

In [9]:
# Let's call K-Fold:
# n_splits specify how many fold do you want?
from sklearn.model_selection import KFold
kf = KFold(n_splits = 3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [11]:
# Now the folds are created. The way we use this K-fold on the datasets is:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


* So as we see the sample dataset[1-9] is splited into three folds. in First iteration it used first fold [0 1 2] for testing and remaining [3 4 5 6 7 8 ] for training. in the second iteration it used [3 4 5] for testing and [0 1 2 6 7 8] for training. and similar in 3rd iteration. 
* So now we use K-fold for our Digits example.

In [12]:
# To simplify the things, we create a generic method which takes a model, X_train, X_test, y_train and y_test as input.
# And then train the corresponding model: